# Configuration de Cuda dans Google Colab

In [ ]:
!nvcc -V

In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-vhp9n03l
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-vhp9n03l
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=9307ebc265eb3c3c277a0a9c8632eb909bb66f032d97b953d5a3ca00e5dbf81e
  Stored in directory: /tmp/pip-ephem-wheel-cache-pa486kt2/wheels/db/c1/1f/a2bb07bbb4a1ce3c43921252aeafaa6205f08637e292496f04
Successfully built NVCCPlugin


## On vérifie que l'on est bien connecté au GPU

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

## Chargement du plugin nvcc permettant de compiler/executer les programmes Cuda

In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


## Un makefile est déjà à votre disposition pour compiler les programme du TP


### 1. Executez la cellule du Makefile

Le makefile a été modifié pour les programmes puisse s'exécuter avec la GPU premium.



In [ ]:
%%writefile Makefile
# Change the example variable to build a different source module (e.g. EXAMPLE=exercice01)
EXAMPLE=program

# Makefile variables 
# Add extra targets to OBJ with space separator e.g. If there is as source file random.c then add random.o to OBJ)
# Add any additional dependancies (header files) to DEPS. e.g. if there is aheader file random.h required by your source modules then add this to DEPS.
CC=gcc
CFLAGS= -O3 -Wextra -fopenmp
NVCC=nvcc
NVCC_FLAGS= -gencode arch=compute_75,code=sm_75 -gencode arch=compute_80,code=sm_80 -gencode arch=compute_86,code=sm_86 -gencode arch=compute_87,code=sm_87
OBJ=$(EXAMPLE).o
DEPS=

# Build rule for object files ($@ is left hand side of rule, $< is first item from the right hand side of rule)
%.o : %.cu $(DEPS)
	$(NVCC) -c -o $@ $< $(NVCC_FLAGS) $(addprefix -Xcompiler ,$(CCFLAGS))

# Make example ($^ is all items from right hand side of the rule)
$(EXAMPLE) : $(OBJ)
	$(NVCC) -o $@ $^ $(NVCC_FLAGS) $(addprefix -Xcompiler ,$(CCFLAGS))

# PHONY prevents make from doing something with a filename called clean
.PHONY : clean
clean:
	rm -rf $(EXAMPLE) $(OBJ)

Writing Makefile


# TP4


In [ ]:
%%writefile utils.h
#ifndef __UTILS_H__
#define __UTILS_H__
#include <stdio.h>

static void HandleError( cudaError_t err,
                         const char *file,
                         int line ) {
    if (err != cudaSuccess) {
        printf( "%s in %s at line %d\n", cudaGetErrorString( err ),
                file, line );
        exit( EXIT_FAILURE );
    }
}
#define HANDLE_ERROR( err ) (HandleError( err, __FILE__, __LINE__ ))


#define HANDLE_NULL( a ) {if (a == NULL) { \
                            printf( "Host memory failed in %s at line %d\n", \
                                    __FILE__, __LINE__ ); \
                            exit( EXIT_FAILURE );}}

template< typename T >
void swap( T& a, T& b ) {
    T t = a;
    a = b;
    b = t;
}


void* big_random_block( int size ) {
    unsigned char *data = (unsigned char*)malloc( size );
    HANDLE_NULL( data );
    for (int i=0; i<size; i++)
        data[i] = rand();

    return data;
}

int* big_random_block_int( int size ) {
    int *data = (int*)malloc( size * sizeof(int) );
    HANDLE_NULL( data );
    for (int i=0; i<size; i++)
        data[i] = rand();

    return data;
}


// a place for common kernels - starts here

__device__ unsigned char value( float n1, float n2, int hue ) {
    if (hue > 360)      hue -= 360;
    else if (hue < 0)   hue += 360;

    if (hue < 60)
        return (unsigned char)(255 * (n1 + (n2-n1)*hue/60));
    if (hue < 180)
        return (unsigned char)(255 * n2);
    if (hue < 240)
        return (unsigned char)(255 * (n1 + (n2-n1)*(240-hue)/60));
    return (unsigned char)(255 * n1);
}

__global__ void float_to_color( unsigned char *optr,
                              const float *outSrc ) {
    // map from threadIdx/BlockIdx to pixel position
    int x = threadIdx.x + blockIdx.x * blockDim.x;
    int y = threadIdx.y + blockIdx.y * blockDim.y;
    int offset = x + y * blockDim.x * gridDim.x;

    float l = outSrc[offset];
    float s = 1;
    int h = (180 + (int)(360.0f * outSrc[offset])) % 360;
    float m1, m2;

    if (l <= 0.5f)
        m2 = l * (1 + s);
    else
        m2 = l + s - l * s;
    m1 = 2 * l - m2;

    optr[offset*4 + 0] = value( m1, m2, h+120 );
    optr[offset*4 + 1] = value( m1, m2, h );
    optr[offset*4 + 2] = value( m1, m2, h -120 );
    optr[offset*4 + 3] = 255;
}

__global__ void float_to_color( uchar4 *optr,
                              const float *outSrc ) {
    // map from threadIdx/BlockIdx to pixel position
    int x = threadIdx.x + blockIdx.x * blockDim.x;
    int y = threadIdx.y + blockIdx.y * blockDim.y;
    int offset = x + y * blockDim.x * gridDim.x;

    float l = outSrc[offset];
    float s = 1;
    int h = (180 + (int)(360.0f * outSrc[offset])) % 360;
    float m1, m2;

    if (l <= 0.5f)
        m2 = l * (1 + s);
    else
        m2 = l + s - l * s;
    m1 = 2 * l - m2;

    optr[offset].x = value( m1, m2, h+120 );
    optr[offset].y = value( m1, m2, h );
    optr[offset].z = value( m1, m2, h -120 );
    optr[offset].w = 255;
}


#if _WIN32
    //Windows threads.
    #include <windows.h>

    typedef HANDLE CUTThread;
    typedef unsigned (WINAPI *CUT_THREADROUTINE)(void *);

    #define CUT_THREADPROC unsigned WINAPI
    #define  CUT_THREADEND return 0

#else
    //POSIX threads.
    #include <pthread.h>

    typedef pthread_t CUTThread;
    typedef void *(*CUT_THREADROUTINE)(void *);

    #define CUT_THREADPROC void
    #define  CUT_THREADEND
#endif

//Create thread.
CUTThread start_thread( CUT_THREADROUTINE, void *data );

//Wait for thread to finish.
void end_thread( CUTThread thread );

//Destroy thread.
void destroy_thread( CUTThread thread );

//Wait for multiple threads.
void wait_for_threads( const CUTThread *threads, int num );

#if _WIN32
    //Create thread
    CUTThread start_thread(CUT_THREADROUTINE func, void *data){
        return CreateThread(NULL, 0, (LPTHREAD_START_ROUTINE)func, data, 0, NULL);
    }

    //Wait for thread to finish
    void end_thread(CUTThread thread){
        WaitForSingleObject(thread, INFINITE);
        CloseHandle(thread);
    }

    //Destroy thread
    void destroy_thread( CUTThread thread ){
        TerminateThread(thread, 0);
        CloseHandle(thread);
    }

    //Wait for multiple threads
    void wait_for_threads(const CUTThread * threads, int num){
        WaitForMultipleObjects(num, threads, true, INFINITE);

        for(int i = 0; i < num; i++)
            CloseHandle(threads[i]);
    }

#else
    //Create thread
    CUTThread start_thread(CUT_THREADROUTINE func, void * data){
        pthread_t thread;
        pthread_create(&thread, NULL, func, data);
        return thread;
    }

    //Wait for thread to finish
    void end_thread(CUTThread thread){
        pthread_join(thread, NULL);
    }

    //Destroy thread
    void destroy_thread( CUTThread thread ){
        pthread_cancel(thread);
    }

    //Wait for multiple threads
    void wait_for_threads(const CUTThread * threads, int num){
        for(int i = 0; i < num; i++)
            end_thread( threads[i] );
    }

#endif

#endif  // __UTILS_H__

Writing utils.h


## 1. Mémoire paginée vs. Mémoire épinglée.

Afin de visualiser le gain que l'on obtient en utilisant la mémoire épinglée, vous allez implémentez deux fonctions **cuda_malloc(int n, bool up)** et **cuda_host_alloc(int n, bool up)**



### cuda_malloc() 
#### 1.1 Allouez la mémoire hôte pour le vecteur a à l'aide de malloc
#### 1.2 Allouez la mémoire GPU pour le vecteur dev_A à l'aide de cudaMalloc
#### 1.3 Ecrire une boucle pour i allant de 0 à 100 :
  - si up == True alors copiez le vecteur a dans dev_a
  - sinon copiez le vecteur dev_a dans a.

#### 1.4 Libérez la mémoire.

### cuda_host_alloc()
#### 1.4 Allouez la mémoire hôte pour le vecteur a à l'aide de malloc
#### 1.5 Allouez la mémoire GPU pour le vecteur dev_A à l'aide de cudaMalloc
#### 1.6 Ecrire une boucle pour i allant de 0 à 100 :
  - si up == True alors copiez le vecteur a dans dev_a
  - sinon copiez le vecteur dev_a dans a.

#### 1.7 Libérez la mémoire.

Constatez-vous une différence entre la mémoire paginée et la mémoire épinglée ?


In [ ]:
%%writefile program.cu
#include "utils.h"

#define N  (64*1024*1024)


float cuda_malloc( int n, bool up ) {
    cudaEvent_t     start, stop;
    int             *a, *dev_a;
    float           elapsedTime;

    HANDLE_ERROR( cudaEventCreate( &start ) );
    HANDLE_ERROR( cudaEventCreate( &stop ) );

    // 1.1 Allouez de la mémoire hôte pour le vecteur a de taille n (malloc)
    HANDLE_NULL( a );
    // 1.2 Allouez de la mémoire GPU pour dev_a de taille n

    HANDLE_ERROR( cudaEventRecord( start, 0 ) );
    // 1.3 Pour i allant de 0 à 100 : si up == True alors copiez le vecteur a dans dev_a, sinon copiez le vecteur dev_a dans a.
    
    
    HANDLE_ERROR( cudaEventRecord( stop, 0 ) );
    HANDLE_ERROR( cudaEventSynchronize( stop ) );
    HANDLE_ERROR( cudaEventElapsedTime( &elapsedTime,
                                        start, stop ) );

    // 1.4 Libérez la mémoire.
    HANDLE_ERROR( cudaEventDestroy( start ) );
    HANDLE_ERROR( cudaEventDestroy( stop ) );

    return elapsedTime;
}


float cuda_host_alloc( int n, bool up ) {
    cudaEvent_t     start, stop;
    int             *a, *dev_a;
    float           elapsedTime;

    HANDLE_ERROR( cudaEventCreate( &start ) );
    HANDLE_ERROR( cudaEventCreate( &stop ) );

    // 1.5 Allouez de la mémoire hôte dans la mémoire épinglée pour le vecteur a de taille n (cudaHostAlloc)
    // 1.6 Allouez de la mémoire GPU pour dev_a de taille n

    HANDLE_ERROR( cudaEventRecord( start, 0 ) );
    
    // 1.7 Pour i allant de 0 à 100 : si up == True alors copiez le vecteur a dans dev_a, sinon copiez le vecteur dev_a dans a.
    HANDLE_ERROR( cudaEventRecord( stop, 0 ) );
    HANDLE_ERROR( cudaEventSynchronize( stop ) );
    HANDLE_ERROR( cudaEventElapsedTime( &elapsedTime,
                                        start, stop ) );

    // 1.8 Libérez la mémoire.
    HANDLE_ERROR( cudaEventDestroy( start ) );
    HANDLE_ERROR( cudaEventDestroy( stop ) );

    return elapsedTime;
}


int main( void ) {
    float           elapsedTime;
    float           MB = (float)100*N*sizeof(int)/1024/1024;


    // try it with cudaMalloc
    elapsedTime = cuda_malloc_test( N, true );
    printf( "Time using cudaMalloc:  %3.1f ms\n",
            elapsedTime );
    printf( "\tMB/s during copy up:  %3.1f\n",
            MB/(elapsedTime/1000) );

    elapsedTime = cuda_malloc_test( N, false );
    printf( "Time using cudaMalloc:  %3.1f ms\n",
            elapsedTime );
    printf( "\tMB/s during copy down:  %3.1f\n",
            MB/(elapsedTime/1000) );

    // now try it with cudaHostAlloc
    elapsedTime = cuda_host_alloc_test( N, true );
    printf( "Time using cudaHostAlloc:  %3.1f ms\n",
            elapsedTime );
    printf( "\tMB/s during copy up:  %3.1f\n",
            MB/(elapsedTime/1000) );

    elapsedTime = cuda_host_alloc_test( N, false );
    printf( "Time using cudaHostAlloc:  %3.1f ms\n",
            elapsedTime );
    printf( "\tMB/s during copy down:  %3.1f\n",
            MB/(elapsedTime/1000) );
}

Writing program.cu


In [ ]:
!make

In [ ]:
!./program

## 2. Streams et execution asynchrone

Afin de visualiser le gain que l'on obtient en utilisant la mémoire épinglée, vous allez implémentez deux fonctions **cuda_malloc(int n, bool up)** et **cuda_host_alloc(int n, bool up)**



### Mémoire paginée vs. épinglée
  Pour le moment la mémoire pour les vecteurs host h_x, h_y et h_y1 est alloué via la fonction malloc. La mémoire alloué par malloc est paginable.
  
#### 2.1 Compilez , executez le programme et relevez le temps d'execution en sortie du programme.
#### 2.2 Modifiez le code de sorte à n'utiliser que de la mémoire épingler via cudaHostMalloc(). 
N'oubliez pas de changer la façon dont on libère la mémoire en fin de programme.

Executez à nouveau le programme, que constatez-vous sur le temps d'exécution ?

### Streams
Vous allez maintenant modifier le programme afin de rendre possible l'execution asynchrone entre les copies CPU <--> GPU et les kernels.

Le but etant de diviser le vecteurs 1D h_x en un sous_ensemble de vecteurs definit par la variable constante subpart. Un stream s'occupera d'un sous-vecteur à la fois.

```c
typedef float ft;
const int sub_parts = 64;
const size_t ds = 1024*1024*sub_parts;
const int count = 22;
const int num_streams = 8;
```

Dans le code, differentes constantes ont été déclarés:

- **ft** : type utilisé pour declarer nos variables dans le code (changer float par double pour utiliser la précision double)
- **sub_paths** : Permet de diviser notre vecteur en un sous-ensemble de vecteurs.
- **ds** : Correspond à la taille totale du vecteur. Dans la partie streams du TP, sub_parts permet d'avoir 64 vecteurs de taille 1024*1024 
- **count** : Permet de définir la taille de l'interval de valeurs utilisé pour calculer la moyenne de la probabilité de densité.
- **num_streams** : Correspond au nombre de streams que l'on veut lancer.

Vous ecrirez le code correspondant aux streams dans la partie délimité par **#ifdef USE_STREAM #endif**

```c
#ifdef USE_STREAMS

	// Code correspondant aux streams

#endif
```

#### 2.3 Creation des streams

Ecrivez le ou les instructions code permettant de creer vos streams de tailles num_streams en utilisant la fonction **cudaStreamCreate()**

#### 2.4 Destruction des streams

Ecrivez le ou les instructions permettant de détruire vos streams en utilisant la fonction **cudaStreamDestroy()**.

#### 2.5 Execution des streams

Pour chaque stream : 

- Faite une copie asynchrone CPU vers GPU du vecteur **h_x** dans **d_x** via **cudaMemcpyAsync()**.
- Lancer le kernel pour le stream courant
- Faite la copie GPU vers CPU du résultat **d_y** dans **h_y** via **cudaMemcpyAsync()**.

Pour rappel, chaque streams s'occupe d'un sous-ensemble du vecteur **h_x**. 

Si **h_x** est divisé en 64 sous-vecteurs et nous n'avons que 8 streams alors les treams 0, 1, 2, ..., 7 s'occuperont respectivement des sous-vecteurs 0, 1, 2, ... 7 puis des sous-vecteurs 8, 9, 10, ..., 15 et ainsi de suite... 
Pour les streams pensez à utiliser un offset, pour retourver l'id du stream vous pouvez utiliser le module % :

- 0%4 = 0, 1%4 = 1, 2%4 = 2, 3%4 = 3
- 4%4 = 0, 5%4 = 1, 6%4 = 2, 7%4 = 3

Chaque sous-vecteur est de taille 1024x1024, donc : 

- Le stream 0 s'occupera du sous-vecteur 0 de taille 1024x1024 commençant par l'indice 0.
- Le stream 1 s'occupera du sous-vecteur 1 de taille 1024x1024 commençant par l'indice (1024x1024).
- Le stream 2 s'occupera du sous-vecteur 2 de taille 1024x1024 commençant par l'indice 2*(1024x1024).

Si vous exécuté le code, une étape de vérification sera effectué pour s'assurer que votre implémentation est correcte.

#### 2.6 Execution asynchrone

Pour executer le code en mode streams, c'est à dire la partie délimité par **#ifdef USE_STREAM #endif**, rajouter le flag -DUSE_STREAMS dans le makefile.

```c
NVCC_FLAGS = -DUSE_STREAMS -gencode arch=compute_75,code=sm_75 -gencode arch=compute_80,code=sm_80 -gencode arch=compute_86,code=sm_86 -gencode arch=compute_87,code=sm_87
```
Executez le code et comparez les temps d'execution non-streams vs. streams.

Que constatez-vous ?

#### 2.7 Modifiez les paramètres
```c
typedef float ft;
const int sub_parts = 64;
const size_t ds = 1024*1024*sub_parts;
const int count = 22;
const int num_streams = 8;
```
Que se passe t'il au niveau du temps d'execution lorsque vous :

1. Changez **typedef float ft** par **typedef double ft** ?
2. Augmentez ou diminuez de manière considerable **sub_parts** ?
3. Changez la taille **ds** ?
3. Augmentez ou diminuez **count** ? 
4. Augmentez ou diminuez **num_streams** ? (nombre limité par GPU)



In [ ]:
%%writefile program.cu
#include <math.h>
#include <iostream>
#include <time.h>
#include <sys/time.h>
#include <stdio.h>

typedef float ft;
const int sub_parts = 64;
const size_t ds = 1024*1024*sub_parts;
const int count = 22;
const int num_streams = 8;

const float sqrt_2PIf = 2.5066282747946493232942230134974f;
const double sqrt_2PI = 2.5066282747946493232942230134974;
__device__ float gpdf(float val, float sigma) {
  return expf(-0.5f * val * val) / (sigma * sqrt_2PIf);
}

__device__ double gpdf(double val, double sigma) {
  return exp(-0.5 * val * val) / (sigma * sqrt_2PI);
}

//  calcul la moyenne de la densite de probabilite sur un interval de valeurs autour de chaque point.
__global__ void gaussian_pdf(const ft * __restrict__ x, ft * __restrict__ y, const ft mean, const ft sigma, const int n) {
  int idx = threadIdx.x + blockDim.x * blockIdx.x;
  if (idx < n) {
    ft in = x[idx] - (count / 2) * 0.01f;
    ft out = 0;
    for (int i = 0; i < count; i++) {
      ft temp = (in - mean) / sigma;
      out += gpdf(temp, sigma);
      in += 0.01f;
    }
    y[idx] = out / count;
  }
}

// Verification d'erreur CUDA
#define cudaCheckErrors(msg) \
  do { \
    cudaError_t __err = cudaGetLastError(); \
    if (__err != cudaSuccess) { \
        fprintf(stderr, "Fatal error: %s (%s at %s:%d)\n", \
            msg, cudaGetErrorString(__err), \
            __FILE__, __LINE__); \
        fprintf(stderr, "*** FAILED - ABORTING\n"); \
        exit(1); \
    } \
  } while (0)

// Calcul du temps sur l'host
#define USECPSEC 1000000ULL

unsigned long long dtime_usec(unsigned long long start) {
  timeval tv;
  gettimeofday(&tv, 0);
  return ((tv.tv_sec*USECPSEC)+tv.tv_usec)-start;
}

int main() {
  ft *h_x, *d_x, *h_y, *h_y1, *d_y;
  h_x = (ft *)malloc(ds*sizeof(ft));
  h_y = (ft *)malloc(ds*sizeof(ft));
  h_y1 = (ft *)malloc(ds*sizeof(ft));

  cudaMalloc(&d_x, ds*sizeof(ft));
  cudaMalloc(&d_y, ds*sizeof(ft));
  cudaCheckErrors("allocation error");

  gaussian_pdf<<<(ds + 255) / 256, 256>>>(d_x, d_y, 0.0, 1.0, ds); // warm-up

  for (size_t i = 0; i < ds; i++) {
    h_x[i] = rand() / (ft)RAND_MAX;
  }
  cudaDeviceSynchronize();

  unsigned long long et1 = dtime_usec(0);

  cudaMemcpy(d_x, h_x, ds * sizeof(ft), cudaMemcpyHostToDevice);
  gaussian_pdf<<<(ds + 255) / 256, 256>>>(d_x, d_y, 0.0, 1.0, ds);
  cudaMemcpy(h_y1, d_y, ds * sizeof(ft), cudaMemcpyDeviceToHost);
  cudaCheckErrors("non-streams execution error");

  et1 = dtime_usec(et1);
  std::cout << "non-stream elapsed time: " << et1/(float)USECPSEC << std::endl;

#ifdef USE_STREAMS
  cudaMemset(d_y, 0, ds * sizeof(ft));

  unsigned long long et = dtime_usec(0);

  // 2.3 Creation des streams
 

  // 2.5 Execution des streams
  


  et = dtime_usec(et);

  for (int i = 0; i < ds; i++) {
    if (h_y[i] != h_y1[i]) {
      std::cout << "mismatch at " << i << " was: " << h_y[i] << " should be: " << h_y1[i] << std::endl;
      return -1;
    }
  }

  // 2.4 Destruction des streams

  std::cout << "streams elapsed time: " << et/(float)USECPSEC << std::endl;
#endif

  return 0;
}